In [376]:
import pandas as pd
import numpy as np
import string
import re
import nltk

In [377]:
df = pd.read_csv('DataScientist.csv')
df = df.drop("Job Description", axis=1)
df = df.drop("Unnamed: 0", axis=1)


In [378]:
for i in df.index:
    if df.loc[i,"Easy Apply"] == "-1":
        df.loc[i,"Easy Apply"] = False
df["Easy Apply"]=df["Easy Apply"].astype(bool)

In [379]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              3909 non-null   int64  
 1   Job Title          3909 non-null   object 
 2   Salary Estimate    3909 non-null   object 
 3   Rating             3909 non-null   float64
 4   Company Name       3909 non-null   object 
 5   Location           3909 non-null   object 
 6   Headquarters       3909 non-null   object 
 7   Size               3909 non-null   object 
 8   Founded            3909 non-null   int64  
 9   Type of ownership  3909 non-null   object 
 10  Industry           3909 non-null   object 
 11  Sector             3909 non-null   object 
 12  Revenue            3909 non-null   object 
 13  Competitors        3909 non-null   object 
 14  Easy Apply         3909 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(11)
memory usage: 431.5+ KB


In [380]:
#data cleaning
df.replace(['-1'], [np.nan], inplace=True)
df.replace(['-1.0'], [np.nan], inplace=True)
df.replace([-1], [np.nan], inplace=True)

In [381]:
df.isnull().sum()

#df.info()

index                   0
Job Title               0
Salary Estimate         0
Rating                409
Company Name            0
Location                0
Headquarters          240
Size                  229
Founded               977
Type of ownership     229
Industry              546
Sector                546
Revenue               229
Competitors          2760
Easy Apply              0
dtype: int64

In [382]:
#seperating salary estimate into min and max salary 
df_salary = df["Salary Estimate"].str.split("-",expand=True)
minimum_salary = df_salary[0]
minimum_salary = minimum_salary.str.replace("K"," ")
maximum_salary = df_salary[1].str.replace('(Glassdoor est.)', ' ',regex=True)
maximum_salary = maximum_salary.str.replace('(', ' ',regex=True)
maximum_salary = maximum_salary.str.replace(')', ' ',regex=True)
maximum_salary = maximum_salary.str.replace('K', ' ',regex=True)
maximum_salary = maximum_salary.str.replace('Employer est.', ' ',regex=True)
maximum_salary = maximum_salary.str.replace('Per Hour', ' ',regex=True)

maximum_salary = maximum_salary.str.replace('$', ' ',regex=True).fillna(0).astype(int)
minimum_salary = minimum_salary.str.replace('$', ' ',regex=True).fillna(0).astype(int)
maximum_salary.value_counts()




144    120
175     93
151     92
140     90
122     90
      ... 
53       7
185      7
173      6
26       3
74       1
Name: 1, Length: 97, dtype: int64

In [383]:
df['Minimum Salary'] = minimum_salary
df['Maximum Salary'] = maximum_salary

df.drop('Salary Estimate',axis = 1,inplace = True)
df['Company Name'] = df['Company Name'].str.replace('\n.*', ' ',regex=True)
df['Est_Salary']= (df['Minimum Salary']+df['Maximum Salary'])/2


In [384]:
#seperating location and HQ

Location = df["Location"].str.split(",",expand=True)
Location_city = Location[0]
Location_state = Location[1]
df["Location City"] = Location_city
df["Location State"] = Location_state
df.drop("Location",axis = 1, inplace=True)

HQ = df["Headquarters"].str.split(",",expand=True)
HQ_city = HQ[0]
HQ_state = HQ[1]
df["HQ City"] = HQ_city
df["HQ State"] = HQ_state
df.drop("Headquarters",axis = 1, inplace=True)



In [385]:
#seperating job tilte from department

department = df["Job Title"].str.split(",", expand=True)
df["Job Title"], df["Department"] = department[0], department[1]
df.drop("Department",axis=1,inplace=True)

In [386]:
df["Job Title"].value_counts()

Data Scientist                                       335
Data Engineer                                        268
Data Analyst                                         264
Senior Data Scientist                                104
Senior Data Analyst                                   52
                                                    ... 
Principal Device Modeling Engineer                     1
SQL/SAS Data Analyst                                   1
Marketing Data Analyst Coordinator                     1
Greenplum /Big Data Engineer/Architect                 1
Patient Safety Physician or Safety Scientist - UK      1
Name: Job Title, Length: 1797, dtype: int64

In [387]:
df["Job Title"] = df["Job Title"].str.replace(" â","")


In [388]:
df["Revenue"].value_counts()

Unknown / Non-Applicable            1163
$10+ billion (USD)                   599
$100 to $500 million (USD)           338
$50 to $100 million (USD)            214
$2 to $5 billion (USD)               211
$10 to $25 million (USD)             191
$1 to $2 billion (USD)               180
$1 to $5 million (USD)               170
$25 to $50 million (USD)             143
$5 to $10 billion (USD)              133
Less than $1 million (USD)           124
$500 million to $1 billion (USD)     114
$5 to $10 million (USD)              100
Name: Revenue, dtype: int64

In [389]:
df["Revenue"] = df["Revenue"].replace("Unknown / Non-Applicable",np.nan)


In [390]:
df['Revenue'] = df['Revenue'].str.replace('$', ' ',regex=True)
df['Revenue'] = df['Revenue'].str.replace('(USD)', ' ',regex=True)
df['Revenue'] = df['Revenue'].str.replace('(', ' ',regex=True)
df['Revenue'] = df['Revenue'].str.replace(')', ' ',regex=True)
df['Revenue'] = df['Revenue'].str.replace(' ', '',regex=True)
df['Revenue'] = df['Revenue'].str.replace('+', '',regex=True)

In [391]:
df["Revenue"].value_counts()

10billion               599
100to500million         338
50to100million          214
2to5billion             211
10to25million           191
1to2billion             180
1to5million             170
25to50million           143
5to10billion            133
Lessthan1million        124
500millionto1billion    114
5to10million            100
Name: Revenue, dtype: int64

In [392]:
df['Revenue'] = df['Revenue'].str.replace('2to5billion', '2billionto5billion')
df['Revenue'] = df['Revenue'].str.replace('5to10billion ', '5billionto10billion')

In [393]:
df['Revenue'] = df['Revenue'].replace('million', ' ')
df['Revenue'] = df['Revenue'].replace('10billion', '10billionto11billion')
df['Revenue'] = df['Revenue'].str.replace('Lessthan1million', '0millionto1million')
df['Revenue'] = df['Revenue'].str.replace('million', ' ')
df['Revenue'] = df['Revenue'].str.replace('billion', '000 ')


In [394]:
df["Revenue"].value_counts()

10000 to11000     599
100to500          338
50to100           214
2000 to5000       211
10to25            191
1to2000           180
1to5              170
25to50            143
5to10000          133
0 to1             124
500 to1000        114
5to10             100
Name: Revenue, dtype: int64

In [395]:
df["Revenue"] = df["Revenue"].str.replace(" to","to")


In [396]:
df["Revenue"].value_counts()

10000to11000     599
100to500         338
50to100          214
2000to5000       211
10to25           191
1to2000          180
1to5             170
25to50           143
5to10000         133
0to1             124
500to1000        114
5to10            100
Name: Revenue, dtype: int64

In [397]:
revenue = df["Revenue"].str.split("to",expand=True)

In [398]:
df["max revenue (milions)"] = revenue[1]
df["min revenue (milions)"] = revenue[0]

In [399]:
df.drop("Revenue",axis=1,inplace=True)

In [400]:
df["HQ State"] = df["HQ State"].str.replace("Sweden","SWE")
df["HQ State"] = df["HQ State"].str.replace("Belgium","BE")
df["HQ State"] = df["HQ State"].str.replace("Iran","IR")
df["HQ State"] = df["HQ State"].str.replace("United Kingdom","UK")

In [401]:
df["Founded"] = df["Founded"].astype("Int32")


In [402]:
df["Job Title"] = df["Job Title"].str.replace("/","|")

In [403]:
df["Company Name"] = df["Company Name"].astype(str)


In [404]:
for i in df.index:
    df.loc[i,"index"] = i
        

In [405]:
df["Type of ownership"] = df["Type of ownership"].str.replace("/","|")

In [406]:
df["Size"] = df["Size"].str.replace("employees","",regex=True)
df["Size"] = df["Size"].str.replace("+","",regex=True)
df["Size"] = df["Size"].str.replace(" ","",regex=True)


In [407]:
df["Size"].value_counts()

10000          993
51to200        563
1001to5000     553
1to50          550
201to500       418
501to1000      307
5001to10000    219
Unknown         77
Name: Size, dtype: int64

In [408]:
size = df["Size"].str.split("to",expand=True)
df["min_size_employees"], df["max_size_employees"] = size[0], size[1]


In [409]:
df.drop("Size",axis=1,inplace=True)